In [1]:
import os
import dotenv
import httpx
import time
import requests
from httpx_socks import SyncProxyTransport
from tqdm import tqdm

import openai
from automatic_prompt_engineer import generate, evaluate, config, template, data, llm, ape

dotenv.load_dotenv(".env")

True

In [2]:
transport = SyncProxyTransport.from_url(f'socks5://{os.environ["PROXY_USER"]}:{os.environ['PROXY_PASS']}@{os.environ['PROXY']}:{os.environ['PROXY_PORT']}')
http_client = httpx.Client(transport=transport)

client = openai.OpenAI(
    http_client=http_client,
    api_key=os.environ['OPENAI_API_KEY'],
)

In [3]:
words = ["sane", "direct", "informally", "unpopular", "subtractive", "nonresidential",
    "inexact", "uptown", "incomparable", "powerful", "gaseous", "evenly", "formality",
    "deliberately", "off"]
antonyms = ["insane", "indirect", "formally", "popular", "additive", "residential",
    "exact", "downtown", "comparable", "powerless", "solid", "unevenly", "informality",
    "accidentally", "on"]

In [4]:
eval_template = \
"""Instruction: [PROMPT]
Input: [INPUT]
Output: [OUTPUT]"""

demos_template='Input: [INPUT]\nOutput: [OUTPUT]'

prompt_gen_mode = 'forward'
prompt_gen_template = None

config_path = 'configs/accuracy.yaml'
few_shot_data = None
prompt_gen_data = (words[:5], antonyms[:5])
eval_dataset = dataset=(words[5:], antonyms[5:])

In [5]:
prompt_gen_data

(['sane', 'direct', 'informally', 'unpopular', 'subtractive'],
 ['insane', 'indirect', 'formally', 'popular', 'additive'])

In [6]:
prompt_gen_template = ape.get_simple_prompt_gen_template(
    prompt_gen_template, prompt_gen_mode)
# conf = config.simple_config(
#     eval_model, prompt_gen_model, prompt_gen_mode, num_prompts, eval_rounds, prompt_gen_batch_size, eval_batch_size)
# conf = config.update_config(conf, base_conf)
conf = config.load_config(config_path)

# Generate prompts
eval_template = template.EvalTemplate(eval_template)
demos_template = template.DemosTemplate(demos_template)
if prompt_gen_template is None:
    prompt_gen_template = eval_template.convert_to_generation_template()
else:
    prompt_gen_template = template.GenerationTemplate(prompt_gen_template)

if few_shot_data is None:
    few_shot_data = prompt_gen_data

In [7]:
conf

{'generation': {'num_subsamples': 5,
  'num_demos': 5,
  'num_prompts_per_subsample': 10,
  'model': {'name': 'GPT_forward',
   'batch_size': 500,
   'gpt_config': {'model': 'gpt-3.5-turbo',
    'temperature': 0.9,
    'max_tokens': 50,
    'top_p': 0.9,
    'frequency_penalty': 0.0,
    'presence_penalty': 0.0}}},
 'evaluation': {'method': 'accuracy',
  'num_samples': 10,
  'num_few_shot': 5,
  'model': {'name': 'GPT_forward',
   'batch_size': 500,
   'gpt_config': {'model': 'gpt-3.5-turbo',
    'temperature': 0.7,
    'max_tokens': 50,
    'top_p': 1.0,
    'frequency_penalty': 0.0,
    'presence_penalty': 0.0}}},
 'demo': {'model': {'name': 'GPT_forward',
   'batch_size': 500,
   'gpt_config': {'model': 'gpt-3.5-turbo',
    'temperature': 0.7,
    'max_tokens': 200,
    'top_p': 1.0,
    'frequency_penalty': 0.0,
    'presence_penalty': 0.0}}}}

In [8]:
%%time
prompts = generate.generate_prompts(
    prompt_gen_template, demos_template, prompt_gen_data, conf['generation'], client)
prompts

[GPT_forward] Generating 50 completions, split into 1 batches of size 5000


100%|██████████| 1/1 [00:02<00:00,  2.03s/it]

CPU times: user 116 ms, sys: 37.3 ms, total: 153 ms
Wall time: 2.04 s


['The instruction was to create antonyms or opposites for the given words.',
 'The instruction was to change each word from a negative form to its corresponding positive form by adding a prefix or modifying the word in some way.',
 'The instruction was to change each word from having a negative connotation to a positive connotation by adding a prefix or modifying the word.',
 'The instruction was to generate antonyms for each given word.',
 'The instruction was to change the given word from a negative or neutral form to a positive or intensified form by adding a prefix or modifying the word.',
 'The instruction was to find the antonym of each given word.',
 'The instruction was to find the opposite or antonym of the given input word.',
 'convert the words to their opposite forms or antonyms.',
 'The instruction was to find the opposite or antonym of the given word.',
 'The instruction was to form the opposite or contrasting word from the given input.']

In [9]:
prompts = list(set(prompts))

In [10]:
%%time
res = evaluate.evalute_prompts(prompts, eval_template, eval_dataset, demos_template, few_shot_data,
                                   conf['evaluation']['method'], conf['evaluation'], client)

Eval method: accuracy


100it [01:03,  1.59it/s]

CPU times: user 4.11 s, sys: 43 ms, total: 4.15 s
Wall time: 1min 3s


In [11]:
print(res)

acc: prompt
----------------
1.00: convert the words to their opposite forms or antonyms.
1.00: The instruction was to form the opposite or contrasting word from the given input.
0.90: The instruction was to generate antonyms for each given word.
0.90: The instruction was to find the opposite or antonym of the given word.
0.90: The instruction was to find the opposite or antonym of the given input word.
0.90: The instruction was to find the antonym of each given word.
0.90: The instruction was to create antonyms or opposites for the given words.
0.90: The instruction was to change each word from a negative form to its corresponding positive form by adding a prefix or modifying the word in some way.
0.70: The instruction was to change each word from having a negative connotation to a positive connotation by adding a prefix or modifying the word.
0.60: The instruction was to change the given word from a negative or neutral form to a positive or intensified form by adding a prefix or modi